In [34]:
import pandas as pd

import numpy as np

from sklearn.datasets import load_breast_cancer

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix, ConfusionMatrixDisplay

import matplotlib.pyplot as plt

In [35]:
dataset = load_breast_cancer()

### Définition des targets 

In [36]:
X = dataset.data
y = dataset.target

### Split en train et test 

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True, stratify=y)
# Stratify permet de garder le meme nombre de 0 et de 1 dans le dataset.

### Scaling

In [38]:
#Par default commencer par le standard scaler. 
scaler = StandardScaler() 
scaler.fit(X_train)

StandardScaler()

In [39]:
scaled_X_train = scaler.transform(X_train)
scaled_X_test = scaler.transform(X_test)

### Initialise le modèle 

In [40]:
hyperparameters = {'hidden_layer_sizes': [(1,), (3,), (5,), (7,)], 'activation': ['logistic', 'tanh', 'relu', 'identity']}

In [41]:
model = MLPClassifier(hidden_layer_sizes=(10,1,3), activation='tanh', solver='lbfgs', alpha=0.0001, max_iter=10000, random_state=0, max_fun=15000)

In [42]:
gridSearchCV = GridSearchCV(model,
                            hyperparameters,
                            n_jobs=-1,
                            refit=True,
                            cv=3,
                            return_train_score=True)

gridSearchCV.fit(scaled_X_train, y_train)

GridSearchCV(cv=3,
             estimator=MLPClassifier(activation='tanh',
                                     hidden_layer_sizes=(10, 1, 3),
                                     max_iter=10000, random_state=0,
                                     solver='lbfgs'),
             n_jobs=-1,
             param_grid={'activation': ['logistic', 'tanh', 'relu', 'identity'],
                         'hidden_layer_sizes': [(1,), (3,), (5,), (7,)]},
             return_train_score=True)

In [43]:
gridSearchCV.best_params_

# hidden_layer_sizes (x,) nous donne le nombre de neuromes idéal à paramétrer dans le modèle. 

{'activation': 'tanh', 'hidden_layer_sizes': (1,)}

### Evaluation sur train 

In [44]:
y_predict_train = gridSearchCV.best_estimator_.predict(scaled_X_train)

In [45]:
cm = confusion_matrix(y_train, y_predict_train, normalize='true')
cm

array([[0.97058824, 0.02941176],
       [0.00701754, 0.99298246]])

In [46]:
print(classification_report(y_train, y_predict_train))

              precision    recall  f1-score   support

           0       0.99      0.97      0.98       170
           1       0.98      0.99      0.99       285

    accuracy                           0.98       455
   macro avg       0.99      0.98      0.98       455
weighted avg       0.98      0.98      0.98       455



### Evaluation sur test 

In [47]:
y_predict_test = gridSearchCV.best_estimator_.predict(scaled_X_test)

In [48]:
cm = confusion_matrix(y_test, y_predict_test, normalize='true')
cm

array([[0.95238095, 0.04761905],
       [0.02777778, 0.97222222]])

In [49]:
print(classification_report(y_test, y_predict_test))

              precision    recall  f1-score   support

           0       0.95      0.95      0.95        42
           1       0.97      0.97      0.97        72

    accuracy                           0.96       114
   macro avg       0.96      0.96      0.96       114
weighted avg       0.96      0.96      0.96       114

